In [1]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 128
EPOCHS = 20
LSTM_NODES =512
NUM_SENTENCES = 30000
MAX_SENTENCE_LENGTH = 100
MAX_NUM_WORDS = 30000
EMBEDDING_SIZE = 100

In [3]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in open('fra.txt', encoding="utf-8"):
    count += 1
    if count > NUM_SENTENCES:
        break
    if '\t' not in line:
        continue
    input_sentence = line.rstrip().split('\t')[0]
    output = line.rstrip().split('\t')[1]

    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

print("Number of sample input:", len(input_sentences))
print("Number of sample output:", len(output_sentences))
print("Number of sample output input:", len(output_sentences_inputs))

Number of sample input: 30000
Number of sample output: 30000
Number of sample output input: 30000


In [4]:
print("English sentence: ",input_sentences[180])
print("French translation: ",output_sentences[180])

English sentence:  Beat it.
French translation:  Casse-toi de là. <eos>


In [5]:
#tokenize the input sentences(input language) 
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)
print(input_integer_seq[:5])

[[19], [19], [19], [770], [770]]


In [6]:
word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

Total unique words in the input: 4467


In [7]:
max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Length of longest sentence in input: 6


In [8]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[180]:", encoder_input_sequences[180])

encoder_input_sequences.shape: (30000, 6)
encoder_input_sequences[180]: [  0   0   0   0 383   6]


In [9]:
print(word2idx_inputs["join"])
print(word2idx_inputs["us"])

598
62


In [10]:
#tokenize the output sentences(Output language)
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)
print(output_input_integer_seq[:5])

[[2, 61, 6], [2, 816], [2, 639, 6], [2, 1370, 6], [2, 6191]]


In [11]:
word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

Total unique words in the output: 12596


In [12]:
num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Length of longest sentence in the output: 13


In [13]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[180]:", decoder_input_sequences[180])

decoder_input_sequences.shape: (30000, 13)
decoder_input_sequences[180]: [   2 6237    9  153    0    0    0    0    0    0    0    0    0]


In [14]:
print(word2idx_outputs["<sos>"])
print(word2idx_outputs["joignez-vous"])
print(word2idx_outputs["à"])
print(word2idx_outputs["nous."])

2
3890
18
234


In [15]:
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_output_sequences.shape:", decoder_output_sequences.shape)

decoder_output_sequences.shape: (30000, 13)


In [16]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('C:/Users/karthick/Downloads/glove.6B.100d.txt/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [17]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [18]:
print(embeddings_dictionary["ill"])

[ 0.12648    0.1366     0.22192   -0.025204  -0.7197     0.66147
  0.48509    0.057223   0.13829   -0.26375   -0.23647    0.74349
  0.46737   -0.462      0.20031   -0.26302    0.093948  -0.61756
 -0.28213    0.1353     0.28213    0.21813    0.16418    0.22547
 -0.98945    0.29624   -0.62476   -0.29535    0.21534    0.92274
  0.38388    0.55744   -0.14628   -0.15674   -0.51941    0.25629
 -0.0079678  0.12998   -0.029192   0.20868   -0.55127    0.075353
  0.44746   -0.71046    0.75562    0.010378   0.095229   0.16673
  0.22073   -0.46562   -0.10199   -0.80386    0.45162    0.45183
  0.19869   -1.6571     0.7584    -0.40298    0.82426   -0.386
  0.0039546  0.61318    0.02701   -0.3308    -0.095652  -0.082164
  0.7858     0.13394   -0.32715   -0.31371   -0.20247   -0.73001
 -0.49343    0.56445    0.61038    0.36777   -0.070182   0.44859
 -0.61774   -0.18849    0.65592    0.44797   -0.10469    0.62512
 -1.9474    -0.60622    0.073874   0.50013   -1.1278    -0.42066
 -0.37322   -0.50538    0

In [19]:
print(embedding_matrix[539])

[-0.014588    0.057711    0.23784     0.038358    0.40990001  0.075356
  0.56882    -0.43048    -0.83552003  0.067724    0.54229999 -0.52925003
  0.80431002  0.35905001 -0.0044033  -0.87582999  1.07019997 -0.030807
 -0.54617     0.12723     0.062372    0.45129001  0.20299999  0.22989
  0.47604001 -0.65180999 -0.33974001 -1.12179995  1.06330001  0.49349999
 -0.44218001  0.73657    -0.26374    -0.88155001 -0.73110998  0.27689001
 -0.69608998  0.62984997  0.44922999  0.36131001 -0.25182    -0.060487
  0.63139999 -0.51008999  0.75509     0.37737     0.091904   -0.64363003
 -0.047406   -0.68154001 -0.026457   -1.44140005 -0.072266    0.39379001
  0.27090999 -1.23819995 -0.70762998  0.079198    1.1983      1.00880003
  0.14177001 -0.09531     0.24018     0.078879    0.53956997 -0.27958
  0.42635     0.62883002 -0.21539     1.28419995  0.15964     0.33737001
 -0.47059     0.15877999  0.25371999  0.052108   -0.057918    0.37207001
 -0.1701     -0.57551998  0.70279998 -0.37676001  0.31406999  0

In [20]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [24]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='uint8'
)

In [25]:
decoder_targets_one_hot.shape

(30000, 13, 12597)

In [26]:
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [27]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [28]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [29]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [30]:
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
)

Epoch 1/20
211/211 [==============================] - 281s 1s/step - loss: 2.5698 - accuracy: 0.6821 - val_loss: 1.9353 - val_accuracy: 0.7193
Epoch 2/20
211/211 [==============================] - 644s 3s/step - loss: 1.4154 - accuracy: 0.7876 - val_loss: 1.6689 - val_accuracy: 0.7570
Epoch 3/20
211/211 [==============================] - 278s 1s/step - loss: 1.1745 - accuracy: 0.8175 - val_loss: 1.5262 - val_accuracy: 0.7747
Epoch 4/20
211/211 [==============================] - 271s 1s/step - loss: 1.0199 - accuracy: 0.8358 - val_loss: 1.4481 - val_accuracy: 0.7836
Epoch 5/20
211/211 [==============================] - 279s 1s/step - loss: 0.9009 - accuracy: 0.8502 - val_loss: 1.4054 - val_accuracy: 0.7901
Epoch 6/20
211/211 [==============================] - 279s 1s/step - loss: 0.8148 - accuracy: 0.8617 - val_loss: 1.3637 - val_accuracy: 0.7940
Epoch 7/20
211/211 [==============================] - 286s 1s/step - loss: 0.7360 - accuracy: 0.8731 - val_loss: 1.3413 - val_accuracy: 0.7958

In [32]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [33]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [34]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [35]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [36]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [37]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [38]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [39]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [40]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: Go play in traffic.
Response: va au suis !


In [41]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation)

-
Input: Who'll cook?
Response: qui va cuisiner.


In [50]:
from tensorflow.keras.optimizers import Adam

In [43]:
from tensorflow.keras.layers import Bidirectional

In [44]:
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [45]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [46]:
from tensorflow.keras.layers import Dropout
dropout = Dropout(rate=0.3)
decoder_outputs = dropout(decoder_outputs)

In [47]:
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [64]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam

In [68]:
from tensorflow.python.keras.optimizer_v2 import adam 
model3 = Model([encoder_inputs_placeholder,decoder_inputs_placeholder], decoder_outputs)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [69]:
r = model3.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_split=0.2,
)

Epoch 1/20
188/188 [==============================] - 266s 1s/step - loss: 3.3797 - acc: 0.6555 - val_loss: 2.0642 - val_acc: 0.7063
Epoch 2/20
188/188 [==============================] - 260s 1s/step - loss: 1.5325 - acc: 0.7691 - val_loss: 1.7010 - val_acc: 0.7551
Epoch 3/20
188/188 [==============================] - 273s 1s/step - loss: 1.1734 - acc: 0.8151 - val_loss: 1.5207 - val_acc: 0.7767
Epoch 4/20
188/188 [==============================] - 279s 1s/step - loss: 0.9405 - acc: 0.8359 - val_loss: 1.4136 - val_acc: 0.7898
Epoch 5/20
188/188 [==============================] - 258s 1s/step - loss: 0.7535 - acc: 0.8545 - val_loss: 1.3493 - val_acc: 0.8000
Epoch 6/20
188/188 [==============================] - 244s 1s/step - loss: 0.6034 - acc: 0.8721 - val_loss: 1.3063 - val_acc: 0.8067
Epoch 7/20
188/188 [==============================] - 256s 1s/step - loss: 0.4855 - acc: 0.8899 - val_loss: 1.2822 - val_acc: 0.8114
Epoch 8/20
188/188 [==============================] - 253s 1s/step - 

In [70]:
# Save entire model to a HDFS file
model3.save("Encoder-decoder_Model2.h5")